## 一、ChatGPT及所有大模型都有**两大缺陷**：

* **没有最新信息**。大模型的训练周期很长，且更新一次耗资巨大，所以它的知识都是过去的。GPT-3.5 的知识截至 2022 年 1 月，GPT-4 是 2023 年 4 月。
* **没有「真逻辑」** 。它表现出的逻辑、推理，是训练文本的统计规律，而不是真正的逻辑。
* 比如算加法：

  1. 把 100 以内所有加法算式都训练给大模型，ta 就能回答 100 以内的加法算式
  2. 如果问 ta 更大数字的加法，就不一定对了
  3. 因为 ta 并不懂「加法」，只是记住了 100 以内的加法算式的统计规律
  4. Ta 是用字面意义做数学
* 所以：大模型需要连接真实世界，并对接真逻辑系统

## 二、Actions

ChatGPT 官方提供了Actions（以前叫Plugins）来连接真实世界，它的底层技术是**Function Calling**。

> [Actions - OpenAI API](https://platform.openai.com/docs/actions/introduction)


<img src="gpt-actions.png" style="margin-left: 0px" width=900px>

学会Function Calling后，我们就可以用编程的方式将大模型与真实世界进行连接了🎉🎉🎉


## 三、什么是Function Calling？

> [Function Calling - OpenAI API](https://platform.openai.com/docs/guides/function-calling)

* 一个允许大型语言模型（如 GPT ）**在生成文本的过程中调用外部函数或服务的功能。**
* 模型本身不会调用外部函数或服务，它只会返回JSON格式的调用参数，调用的过程会在本地应用中进行。
* **工作机制**

<img src="function-calling-workflow.png" style="margin-left: 0px" width=800px>


### 示例1：调用本地函数

需求：实现一个回答问题的 AI。题目中如果有加法，必须能精确计算。

In [1]:
# 加载环境变量
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import openai
import os
import json

_ = load_dotenv(find_dotenv())  

client = OpenAI()

In [2]:
def get_completion(messages, model="gpt-3.5-turbo-1106"):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7,  
        tools=[{  # 用 JSON 描述函数。可以定义多个。由大模型决定调用谁。也可能都不调用
            "type": "function",
            "function": {
                "name": "sum",
                "description": "加法器，计算一组数的和",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "numbers": {
                            "type": "array",
                            "items": {
                                "type": "number"
                            }
                        }
                    }
                }
            }
        }],
    )
    return response.choices[0].message

In [3]:
from math import *

prompt = "Tell me the sum of 1, 2, 3, 4, 5, 6, 7, 8, 9, 10."
# prompt = "桌上有 2 个苹果，四个桃子和 3 本书，一共有几个水果？"
# prompt = "1+2+3...+99+100"
# prompt = "1024 乘以 1024 是多少？"  # Tools 里没有定义乘法，会怎样？
# prompt = "太阳从哪边升起？"   # 不需要算加法，会怎样？

messages = [
    {"role": "system", "content": "你是一个小学数学老师，你要教学生加法"},
    {"role": "user", "content": prompt}
]
response = get_completion(messages)

# 把大模型的回复加入到对话历史中
messages.append(response)

print("=====GPT回复=====")
print(response)

# 如果返回的是函数调用结果，则打印出来
if response.tool_calls:
    # 是否要调用 sum
    tool_call = response.tool_calls[0]
    if tool_call.function.name == "sum":
        # 调用 sum
        args = json.loads(tool_call.function.arguments)
        result = sum(args["numbers"])
        print("=====函数返回=====")
        print(result)

        # 把函数调用结果加入到对话历史中
        messages.append(
            {
                "tool_call_id": tool_call.id,  # 用于标识函数调用的 ID
                "role": "tool",
                "name": "sum",
                "content": str(result)  # 数值result 必须转成字符串
            }
        )

        # 再次调用大模型
        print("=====最终回复=====")
        print(get_completion(messages).content)

=====GPT回复=====
ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_f1vcfOZzZoG4J44X0G4NBffl', function=Function(arguments='{"numbers":[1,2,3,4,5,6,7,8,9,10]}', name='sum'), type='function')])
=====函数返回=====
55
=====最终回复=====
The sum of 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 is 55.


### 示例2：远程/多 Function 调用

调用高德地图API，查询地址

In [4]:
def get_completion(messages, model="gpt-3.5-turbo-1106"):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,  
        seed=1024,  # 随机种子保持不变，temperature 和 prompt 不变的情况下，输出就会不变
        tool_choice="auto",  # 默认值，由系统自动决定，返回function call还是返回文字回复
        tools=[{
            "type": "function",
            "function": {
                "name": "get_location_coordinate",
                "description": "根据POI名称，获得POI的经纬度坐标",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "POI名称，必须是中文",
                        },
                        "city": {
                            "type": "string",
                            "description": "POI所在的城市名，必须是中文",
                        }
                    },
                    "required": ["location", "city"],
                }
            }
        },
            {
            "type": "function",
            "function": {
                "name": "search_nearby_pois",
                "description": "搜索给定坐标附近的poi",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "longitude": {
                            "type": "string",
                            "description": "中心点的经度",
                        },
                        "latitude": {
                            "type": "string",
                            "description": "中心点的纬度",
                        },
                        "keyword": {
                            "type": "string",
                            "description": "目标poi的关键字",
                        }
                    },
                    "required": ["longitude", "latitude", "keyword"],
                }
            }
        }],
    )
    return response.choices[0].message

In [7]:
import requests

amap_key = "d8c95a47f819b012aef24470403d7747"

def get_location_coordinate(location, city="温州"):
    url = f"https://restapi.amap.com/v5/place/text?key={amap_key}&keywords={location}&region={city}"
    print(url)
    r = requests.get(url)
    result = r.json()
    if "pois" in result and result["pois"]:
        return result["pois"][0]
    return None

def search_nearby_pois(longitude, latitude, keyword):
    url = f"https://restapi.amap.com/v5/place/around?key={amap_key}&keywords={keyword}&location={longitude},{latitude}"
    print(url)
    r = requests.get(url)
    result = r.json()
    ans = ""
    if "pois" in result and result["pois"]:
        for i in range(min(3, len(result["pois"]))):
            name = result["pois"][i]["name"]
            address = result["pois"][i]["address"]
            distance = result["pois"][i]["distance"]
            ans += f"{name}\n{address}\n距离：{distance}米\n\n"
    return ans

In [9]:
prompt = "温州医科大学附属第一医院附近的公交车站"

messages = [
    {"role": "system", "content": "你是一个地图通，你可以找到任何地址。"},
    {"role": "user", "content": prompt}
]
response = get_completion(messages)
messages.append(response)  # 把大模型的回复加入到对话中
print("=====GPT回复=====")
print(response)

# 如果返回的是函数调用结果，则打印出来
while (response.tool_calls is not None):
    # 1106 版新模型支持一次返回多个函数调用请求
    for tool_call in response.tool_calls:
        args = json.loads(tool_call.function.arguments)
        print(args)

        if tool_call.function.name == "get_location_coordinate":
            print("Call: get_location_coordinate")
            result = get_location_coordinate(**args)
        elif tool_call.function.name == "search_nearby_pois":
            print("Call: search_nearby_pois")
            result = search_nearby_pois(**args)

        print("=====函数返回=====")
        print(result)

        messages.append({
            "tool_call_id": tool_call.id,  # 用于标识函数调用的 ID
            "role": "tool",
            "name": tool_call.function.name,
            "content": str(result)  # 数值result 必须转成字符串
        })

    response = get_completion(messages)
    messages.append(response)  # 把大模型的回复加入到对话中

print("=====最终回复=====")
print(response.content)

=====GPT回复=====
ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_XMsBPP6H8g8mEbQUlWsFAQSO', function=Function(arguments='{"location":"温州医科大学附属第一医院","city":"温州"}', name='get_location_coordinate'), type='function')])
{'location': '温州医科大学附属第一医院', 'city': '温州'}
Call: get_location_coordinate
https://restapi.amap.com/v5/place/text?key=d8c95a47f819b012aef24470403d7747&keywords=温州医科大学附属第一医院&region=温州
=====函数返回=====
{'parent': '', 'address': '南白象街道上蔡村', 'distance': '', 'pcode': '330000', 'adcode': '330304', 'pname': '浙江省', 'cityname': '温州市', 'type': '医疗保健服务;综合医院;三级甲等医院', 'typecode': '090101', 'adname': '瓯海区', 'citycode': '0577', 'name': '温州医科大学附属第一医院南白象院区', 'location': '120.695473,27.938175', 'id': 'B0FFGRGJRO'}
{'longitude': '120.695473', 'latitude': '27.938175', 'keyword': '公交车站'}
Call: search_nearby_pois
https://restapi.amap.com/v5/place/around?key=d8c95a47f819b012aef24470403d7747&keywords=公交车站&location=120.695473,27

### 示例3：用 Function Calling 获取 JSON 结构

使用function calling返回结果中的参数，从一段文字中抽取联系人姓名、地址和电话

In [10]:
def get_completion(messages, model="gpt-3.5-turbo-1106"):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,  # 模型输出的随机性，0 表示随机性最小
        tools=[{
            "type": "function",
            "function": {
                "name": "add_contact",
                "description": "添加联系人",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "name": {
                            "type": "string",
                            "description": "联系人姓名"
                        },
                        "address": {
                            "type": "string",
                            "description": "联系人地址"
                        },
                        "tel": {
                            "type": "string",
                            "description": "联系人电话"
                        },
                    }
                }
            }
        }],
    )
    return response.choices[0].message


prompt = "帮我寄给王卓然，地址是北京市朝阳区亮马桥外交办公大楼，电话13012345678。"
messages = [
    {"role": "system", "content": "你是一个联系人录入员。"},
    {"role": "user", "content": prompt}
]
response = get_completion(messages)
print("====GPT回复====")
print(response)
# 使用function calling返回结果中的参数
args = json.loads(response.tool_calls[0].function.arguments)
print("====函数参数====")
print(args)

====GPT回复====
ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_15cfU05mMwui41iLJmhr4JGF', function=Function(arguments='{"name":"王卓然","address":"北京市朝阳区亮马桥外交办公大楼","tel":"13012345678"}', name='add_contact'), type='function')])
====函数参数====
{'name': '王卓然', 'address': '北京市朝阳区亮马桥外交办公大楼', 'tel': '13012345678'}


### 示例4：并行调用

In [9]:
from openai import OpenAI
import os
import json

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())  

client = OpenAI()

def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

def run_conversation():
    # 步骤 1：向模型发送对话和函数定义
    messages = [{"role": "user", "content": "What's the weather like in San Francisco, Tokyo, and Paris?"}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # 步骤 2：检查模型是否要调用函数
    if tool_calls:
        
        available_functions = {
            "get_current_weather": get_current_weather,
        }
        messages.append(response_message)  # 补充上下文
        # 步骤 3: 函数调用
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )
            # 步骤 4：向模型发送每个函数调用和函数响应的信息
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response
reuslt = run_conversation()

In [11]:
reuslt.choices[0].message.content

'The current weather in:\n- San Francisco, CA: 72°C\n- Tokyo, Japan: 10°C\n- Paris, France: 22°C'

## 四、Function Calling 支持情况

* 支持的国外大模型：

  * OpenAI系列：`gpt-4o`, `gpt-4o-2024-05-13`, `gpt-4o-mini`, `gpt-4o-mini-2024-07-18`, `gpt-4-turbo`, `gpt-4-turbo-2024-04-09`, `gpt-4-turbo-preview`, `gpt-4-0125-preview`, `gpt-4-1106-preview`, `gpt-4`, `gpt-4-0613`, `gpt-3.5-turbo`, `gpt-3.5-turbo-0125`, `gpt-3.5-turbo-1106`, `gpt-3.5-turbo-0613`.
  * Gemimi系列：`Gemini 1.5 Pro`, `Gemini 1.5 Flash`
  * Claude系列：`Claude 3 HaiKu`, `Claude 3 Opus`, `Claude3 Sonnet`, `Claude 3.5 Sonnet`
  * ...
* 支持的国内大模型

  * 千问系列：`Qwen-Max`, `Qwen-Plus`, `Qwen-Turbo`
  * 智普系列：`GlM-4`, `GLM-3-turbo`
  * MiniMax系列
  * 百度文心系列
  * ...